In [27]:
# import the necessary libraries

from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
from IPython.display import Markdown


In [2]:
# initialize the environment and client

load_dotenv(override=True)

openai_client = OpenAI()

In [12]:
# read in the linkedin profile

reader = PdfReader("me/linkedin.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)


   
Contact
gautham.pallapa@gmail.com
www.linkedin.com/in/gpallapa
(LinkedIn)
etransforms.com  (Personal)
transformity.info/blog  (Blog)
Top Skills
Technology Management
IT Transformation
Strategic Leadership
Certifications
Certified SAFe® 5 Agilist
Digital Business Transformation
Management
Lean Six Sigma Black Belt
Agile Coach (ICP - ACC)
Career Essentials in Generative AI
by Microsoft and LinkedIn
Honors-Awards
2023 Global Leadership Institute
Award
2022 Global Leadership Institute
Award
2023 Nautilus Award
2023 Book Excellence Award
2021 Outstanding Mentor Award
Publications
Adaptive and Context-Aware Privacy
Preservation Schemes Exploiting
User Interactions in Pervasive
Environments
A Smart Agent based Audio
Authentication Scheme in Ad Hoc
Wireless Networks
User-centric Situation Prediction in
Ubiquitous Computing Environments
Privacy Infusion in Ubiquitous
Computing
A Middleware Framework for
Ambiguous Context Mediation in
Smart Healthcare ApplicationsGautham Pallapa, Ph. D.
Tech

In [13]:
with open("me/summary.txt", "r", encoding="utf-8") as file:
    summary = file.read()

print(summary)

My name is Gautham Pallapa. I am a visionary executive with over 20 years of leadership experience in cloud, data, and AI, driving strategic innovation, operational excellence, and measurable business outcomes. I partner with Fortune 1000 companies to align technology with business goals, delivering significant value through AI-powered solutions, cloud modernization, and data-driven insights. As the award-winning author of "Leading with Empathy", I believe that empathy is a cornerstone of successful leadership. I am also the author of "Mastering Enterprise Platform Engineering". I used to live in the San Francisco Bay Area. I live in the Greater Toronto Area now. 


In [16]:
name = "Gautham Pallapa"

In [17]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
print(system_prompt)

In [21]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [22]:
gr.ChatInterface(chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## What I am going to do next...

1. Ask an LLM to evaluate an answer
2. Rerun if the answer fails evaluation
3. Put this together into 1 workflow

This will currently not use any agentic framework.

In [23]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [24]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [29]:
# display(Markdown(evaluator_system_prompt))

In [30]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [41]:
# Set up the Gemini client

import os

from openai import api_key
gemini_client = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [38]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini_client.beta.chat.completions.parse(
        model="gemini-2.0-flash", 
        messages=messages, 
        response_format=Evaluation)
    return response.choices[0].message.parsed

In [35]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai_client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages
)
reply = response.choices[0].message.content

In [42]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is an acceptable response. It directly answers the user's question with information that is present on the provided LinkedIn profile. The agent also provides context and offers to elaborate on the answer, following the instructions given.")

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [52]:
def chat(message, history):
    if "pineapple" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

Passed evaluation - returning reply


In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The Agent is supposed to be representing Gautham Pallapa in a professional and engaging way. The response uses pig latin, which is unprofessional. 
